In [60]:
import json
import pandas as pd
import os
from collections import defaultdict

In [4]:
with open('../config.json') as config_file:
    config = json.load(config_file)
old_data_path = config['data_path']

print(old_data_path)

/scratch2/hle


In [14]:
# result data from fine tuned detectron
new_data_path = '/scratch2/thdaryan/data/new_result_merge_graph'

In [164]:
df_old = pd.read_csv(os.path.join(old_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_100.tsv'), encoding='utf-8',sep='\t')
df_old.head()

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,ATTR_thin,ATTR_decorative,ATTR_wrinkled,ATTR_peeling,ATTR_golden,ATTR_metallic,ATTR_back,ATTR_thick,ATTR_black and white,ATTR_leaning
0,jeans-1,552272,1206290,2606,0.060614,276.863220,165.345779,112.160248,166.017807,0.000624,...,0.002491,0.002491,0.002492,0.002491,0.002491,0.002491,0.002492,0.002491,0.002491,0.002491
1,people-1,552272,1206290,2606,0.451555,34.083862,94.437332,458.196045,302.747314,0.000624,...,0.002494,0.002494,0.002494,0.002494,0.002495,0.002495,0.002496,0.002494,0.002495,0.002499
2,jeans-2,552272,1206290,2606,0.061767,401.397247,217.888779,129.136322,146.937149,0.000624,...,0.002491,0.002491,0.002491,0.002491,0.002491,0.002491,0.002492,0.002491,0.002491,0.002492
3,man-1,552272,1206290,2606,0.086869,395.227600,115.462044,117.106873,227.878662,0.000624,...,0.002494,0.002493,0.002494,0.002493,0.002493,0.002494,0.002495,0.002493,0.002494,0.002528
4,man-2,552272,1206290,2606,0.082543,254.167130,92.491837,113.958328,222.512070,0.000624,...,0.002494,0.002493,0.002494,0.002493,0.002494,0.002494,0.002496,0.002494,0.002494,0.002530


In [166]:
df_new= pd.read_csv(os.path.join(new_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_100.tsv'), encoding='utf-8',sep='\t')
df_new.head()

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_None-0,...,ATTR_thin,ATTR_decorative,ATTR_wrinkled,ATTR_peeling,ATTR_golden,ATTR_metallic,ATTR_back,ATTR_thick,ATTR_black and white,ATTR_leaning
0,person-1,552272,1206290,2606,0.089289,91.828430,113.562271,91.458847,299.910889,0.010674,...,0.002500,0.002496,0.002498,0.002496,0.002497,0.002497,0.002501,0.002498,0.002500,0.002518
1,person-2,552272,1206290,2606,0.076469,412.426025,133.357635,96.836334,242.586395,0.010675,...,0.002500,0.002496,0.002498,0.002496,0.002496,0.002497,0.002502,0.002498,0.002501,0.002519
2,person-3,552272,1206290,2606,0.077544,528.139648,181.337585,106.816406,223.014313,0.010677,...,0.002499,0.002496,0.002499,0.002496,0.002496,0.002497,0.002505,0.002497,0.002499,0.002519
3,person-4,552272,1206290,2606,0.124209,254.961197,114.384972,156.279587,244.157928,0.010682,...,0.002500,0.002496,0.002502,0.002495,0.002496,0.002497,0.002506,0.002499,0.002500,0.002516
4,person-5,552272,1206290,2606,0.197041,167.750000,104.730003,161.299988,375.269989,0.010676,...,0.002503,0.002496,0.002498,0.002496,0.002497,0.002497,0.002501,0.002499,0.002502,0.002514


In [167]:
df_new.drop(['TYPE_OOV'], axis=1, inplace = True)

In [168]:
# drop type none in new data
list_col = df_new.columns
for col in list_col:
    if(len(col) >= len('TYPE_None') and col[:len('TYPE_None')]=='TYPE_None'):
        df_new.drop([col], axis=1, inplace = True)

In [169]:
list_type_old_data = [col for col in df_old.columns if 'TYPE_' in col]
len(list_type_old_data)

1601

In [170]:
list_type_new_data = [col for col in df_new.columns if 'TYPE_' in col]
len(list_type_new_data)

80

### Make the columns from old df and new df same

In [171]:
# the merged columns will be => type_old_data | type_new_data | attributes

In [172]:
for col in list_type_old_data:
    if (col not in list_type_new_data):
        df_new[col] = 0
        
for col in list_type_new_data:
    if (col not in list_type_old_data):
        df_old[col] = 0

In [173]:
df_new

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_person,...,TYPE_blades,TYPE_towel_rack,TYPE_coaster,TYPE_star,TYPE_petals,TYPE_text,TYPE_feather,TYPE_spots,TYPE_buoy,TYPE_OOV
0,person-1,552272,1206290,2606,0.089289,91.828430,113.562271,91.458847,299.910889,0.028517,...,0,0,0,0,0,0,0,0,0,0
1,person-2,552272,1206290,2606,0.076469,412.426025,133.357635,96.836334,242.586395,0.028361,...,0,0,0,0,0,0,0,0,0,0
2,person-3,552272,1206290,2606,0.077544,528.139648,181.337585,106.816406,223.014313,0.028051,...,0,0,0,0,0,0,0,0,0,0
3,person-4,552272,1206290,2606,0.124209,254.961197,114.384972,156.279587,244.157928,0.027213,...,0,0,0,0,0,0,0,0,0,0
4,person-5,552272,1206290,2606,0.197041,167.750000,104.730003,161.299988,375.269989,0.028212,...,0,0,0,0,0,0,0,0,0,0


In [174]:
df_old

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_baseball glove,TYPE_tennis racket,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier
0,jeans-1,552272,1206290,2606,0.060614,276.863220,165.345779,112.160248,166.017807,0.000624,...,0,0,0,0,0,0,0,0,0,0
1,people-1,552272,1206290,2606,0.451555,34.083862,94.437332,458.196045,302.747314,0.000624,...,0,0,0,0,0,0,0,0,0,0
2,jeans-2,552272,1206290,2606,0.061767,401.397247,217.888779,129.136322,146.937149,0.000624,...,0,0,0,0,0,0,0,0,0,0
3,man-1,552272,1206290,2606,0.086869,395.227600,115.462044,117.106873,227.878662,0.000624,...,0,0,0,0,0,0,0,0,0,0
4,man-2,552272,1206290,2606,0.082543,254.167130,92.491837,113.958328,222.512070,0.000624,...,0,0,0,0,0,0,0,0,0,0
5,woman-1,552272,1206290,2606,0.091059,74.620872,116.848289,108.529228,257.750427,0.000624,...,0,0,0,0,0,0,0,0,0,0
6,shirt-1,552272,1206290,2606,0.029040,90.982697,158.378937,91.706955,97.277191,0.000624,...,0,0,0,0,0,0,0,0,0,0
7,jeans-3,552272,1206290,2606,0.050061,537.248474,256.399078,102.038696,150.715576,0.000624,...,0,0,0,0,0,0,0,0,0,0
8,carpet-1,552272,1206290,2606,0.411775,100.224052,244.056747,539.014221,234.682907,0.000624,...,0,0,0,0,0,0,0,0,0,0
9,floor-1,552272,1206290,2606,0.342651,0.000000,265.571411,492.851105,213.578766,0.000624,...,0,0,0,0,0,0,0,0,0,0


### Merge the old df and new df

In [175]:
# get all class name in old data and save the index
df_merged = df_old.copy()

dict_class_list_index = defaultdict(list)
for i, row in df_old[['box_alias']].iterrows():
    class_name = row['box_alias'][:row['box_alias'].find('-')]
    dict_class_list_index[class_name].append(i)
print('dict_class_list_index from old df: ', dict_class_list_index)

list_idx_to_be_dropped = []
for i, row in df_new.iterrows():
    class_name = row['box_alias'][:row['box_alias'].find('-')]
    
    if (class_name in dict_class_list_index):
        list_idx_to_be_dropped += dict_class_list_index[class_name]
        
    # handle person related class (still hardcoded)
    if (class_name == 'person'):
        list_idx_to_be_dropped += dict_class_list_index['man']
        list_idx_to_be_dropped += dict_class_list_index['woman'] 
    elif (class_name=='man'):
        list_idx_to_be_dropped += dict_class_list_index['man']
    elif (class_name=='woman'):
        list_idx_to_be_dropped += dict_class_list_index['woman']
    
    df_merged = df_merged.append(row, ignore_index = True)
        
set_idx_to_be_dropped = set(list_idx_to_be_dropped)   #just removing some duplicates
print(set_idx_to_be_dropped)

df_merged = df_merged.drop(set_idx_to_be_dropped).reset_index(drop=True)

dict_class_list_index from old df:  defaultdict(<class 'list'>, {'jeans': [0, 2, 7], 'people': [1], 'man': [3, 4, 24], 'woman': [5, 21, 27], 'shirt': [6], 'carpet': [8], 'floor': [9], 'table': [10], 'books': [11], 'dress': [12], 'wall': [13], 'person': [14], 'bookshelf': [15, 17], 'shelf': [16], 'controller': [18], 'tshirt': [19], 'remote': [20], 'shelves': [22], 'men': [23], 'skirt': [25], 'building': [26]})
{3, 4, 5, 14, 21, 24, 27}


In [176]:
df_merged

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_baseball glove,TYPE_tennis racket,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier
0,jeans-1,552272,1206290,2606,0.060614,276.863220,165.345779,112.160248,166.017807,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,people-1,552272,1206290,2606,0.451555,34.083862,94.437332,458.196045,302.747314,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,jeans-2,552272,1206290,2606,0.061767,401.397247,217.888779,129.136322,146.937149,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,shirt-1,552272,1206290,2606,0.029040,90.982697,158.378937,91.706955,97.277191,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,jeans-3,552272,1206290,2606,0.050061,537.248474,256.399078,102.038696,150.715576,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,carpet-1,552272,1206290,2606,0.411775,100.224052,244.056747,539.014221,234.682907,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,floor-1,552272,1206290,2606,0.342651,0.000000,265.571411,492.851105,213.578766,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,table-1,552272,1206290,2606,0.040243,1.153749,368.572815,111.148911,111.225830,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,books-1,552272,1206290,2606,0.080675,374.392334,9.653005,227.808228,108.790878,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,dress-1,552272,1206290,2606,0.070850,163.307434,186.841446,101.489563,214.456619,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [177]:
df_merged['TYPE_person']

0     0.000626
1     0.000626
2     0.000638
3     0.000625
4     0.000704
5     0.000624
6     0.000625
7     0.000639
8     0.000624
9     0.000628
10    0.000625
11    0.000641
12    0.000624
13    0.000624
14    0.000624
15    0.000625
16    0.000624
17    0.000624
18    0.000627
19    0.000628
20    0.000626
21    0.028517
22    0.028361
23    0.028051
24    0.027213
25    0.028212
Name: TYPE_person, dtype: float64

In [178]:
is_nan = False
for col in df_merged.columns:
    is_nan = df_merged[col].isnull().values.any()
print(is_nan)

False


In [179]:
df_merged.to_csv('sample_merged.tsv', sep = '\t', index=False)

In [180]:
df_merged_from_file = pd.read_csv('sample_merged.tsv', encoding='utf-8',sep='\t')
df_merged_from_file

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_baseball glove,TYPE_tennis racket,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier
0,jeans-1,552272,1206290,2606,0.060614,276.863220,165.345779,112.160248,166.017807,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,people-1,552272,1206290,2606,0.451555,34.083862,94.437332,458.196045,302.747314,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,jeans-2,552272,1206290,2606,0.061767,401.397247,217.888779,129.136322,146.937149,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,shirt-1,552272,1206290,2606,0.029040,90.982697,158.378937,91.706955,97.277191,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,jeans-3,552272,1206290,2606,0.050061,537.248474,256.399078,102.038696,150.715576,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,carpet-1,552272,1206290,2606,0.411775,100.224052,244.056747,539.014221,234.682907,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,floor-1,552272,1206290,2606,0.342651,0.000000,265.571411,492.851105,213.578766,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,table-1,552272,1206290,2606,0.040243,1.153749,368.572815,111.148911,111.225830,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,books-1,552272,1206290,2606,0.080675,374.392334,9.653005,227.808228,108.790878,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,dress-1,552272,1206290,2606,0.070850,163.307434,186.841446,101.489563,214.456619,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Wrap all the process into a function

In [186]:
merged_data_path = '/scratch2/thdaryan/data/merged_result'

def merge_old_and_new_df(file_idx):
    df_old = pd.read_csv(os.path.join(old_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), encoding='utf-8',sep='\t')
    df_new = pd.read_csv(os.path.join(new_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), encoding='utf-8',sep='\t')
    
    # Preprocess
    # drop type_oov in df_new because it is already on df_old
    df_new.drop(['TYPE_OOV'], axis=1, inplace = True)
    
    # drop type none in new data
    list_col = df_new.columns
    for col in list_col:
        if(len(col) >= len('TYPE_None') and col[:len('TYPE_None')]=='TYPE_None'):
            df_new.drop([col], axis=1, inplace = True)
    
    list_type_old_data = [col for col in df_old.columns if 'TYPE_' in col]
    list_type_new_data = [col for col in df_new.columns if 'TYPE_' in col]
    
    # add flag to identify it is the old data or new data
    df_old['is_old_data'] = True
    df_new['is_old_data'] = False
    
    # Make the columns in old df and new df same
    for col in list_type_old_data:
        if (col not in list_type_new_data):
            df_new[col] = 0

    for col in list_type_new_data:
        if (col not in list_type_old_data):
            df_old[col] = 0
            
    # Merge old df and new df
    # get all class name in old data and save the index
    df_merged = df_old.copy()

    dict_class_list_index = defaultdict(list)
    for i, row in df_old[['box_alias']].iterrows():
        class_name = row['box_alias'][:row['box_alias'].find('-')]
        dict_class_list_index[class_name].append(i)

    list_idx_to_be_dropped = []
    for i, row in df_new.iterrows():
        class_name = row['box_alias'][:row['box_alias'].find('-')]

        if (class_name in dict_class_list_index):
            list_idx_to_be_dropped += dict_class_list_index[class_name]
        
        # handle person related class (still hardcoded)
        if (class_name == 'person'):
            list_idx_to_be_dropped += dict_class_list_index['man']
            list_idx_to_be_dropped += dict_class_list_index['woman'] 
        elif (class_name=='man'):
            list_idx_to_be_dropped += dict_class_list_index['man']
        elif (class_name=='woman'):
            list_idx_to_be_dropped += dict_class_list_index['woman']

        df_merged = df_merged.append(row, ignore_index = True)

    set_idx_to_be_dropped = set(list_idx_to_be_dropped)   #just removing some duplicates

    df_merged = df_merged.drop(set_idx_to_be_dropped).reset_index(drop=True)
    
    # check if there is NaN value (if so then there is some error on the process)
    is_nan = False
    for col in df_merged.columns:
        is_nan = df_merged[col].isnull().values.any()
    if (is_nan):
        raise Exception('there is NaN value')
        
    # save the result
    df_merged.to_csv(os.path.join(merged_data_path, f'refCOCO/test/attr_tables_with_target_box/attr_{file_idx}.tsv'), sep = '\t', index=False)
    
    

In [187]:
merge_old_and_new_df(100)

In [188]:
pd.read_csv(os.path.join(merged_data_path,f'refCOCO/test/attr_tables_with_target_box/attr_100.tsv'), encoding='utf-8',sep='\t')

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,TYPE_baseball glove,TYPE_tennis racket,TYPE_wine glass,TYPE_hot dog,TYPE_potted plant,TYPE_dining table,TYPE_tv,TYPE_cell phone,TYPE_teddy bear,TYPE_hair drier
0,jeans-1,552272,1206290,2606,0.060614,276.863220,165.345779,112.160248,166.017807,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,people-1,552272,1206290,2606,0.451555,34.083862,94.437332,458.196045,302.747314,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,jeans-2,552272,1206290,2606,0.061767,401.397247,217.888779,129.136322,146.937149,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,shirt-1,552272,1206290,2606,0.029040,90.982697,158.378937,91.706955,97.277191,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,jeans-3,552272,1206290,2606,0.050061,537.248474,256.399078,102.038696,150.715576,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,carpet-1,552272,1206290,2606,0.411775,100.224052,244.056747,539.014221,234.682907,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,floor-1,552272,1206290,2606,0.342651,0.000000,265.571411,492.851105,213.578766,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,table-1,552272,1206290,2606,0.040243,1.153749,368.572815,111.148911,111.225830,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,books-1,552272,1206290,2606,0.080675,374.392334,9.653005,227.808228,108.790878,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,dress-1,552272,1206290,2606,0.070850,163.307434,186.841446,101.489563,214.456619,0.000624,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Process all files

In [ ]:
for i in range(5000):
    merge_old_and_new_df(i)
    if (i % 50 == 0 ):
        print(f'finished processing {i} files')

finished processing 0 files
finished processing 50 files
finished processing 100 files
finished processing 150 files
finished processing 200 files
finished processing 250 files
finished processing 300 files
finished processing 350 files
finished processing 400 files
finished processing 450 files
finished processing 500 files
finished processing 550 files
finished processing 600 files
finished processing 650 files
finished processing 700 files
finished processing 750 files
finished processing 800 files
finished processing 850 files
finished processing 900 files
finished processing 950 files
finished processing 1000 files
finished processing 1050 files
finished processing 1100 files
finished processing 1150 files
finished processing 1200 files
finished processing 1250 files
finished processing 1300 files
finished processing 1350 files
finished processing 1400 files
finished processing 1450 files
finished processing 1500 files
finished processing 1550 files
finished processing 1600 files
